In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set(font_scale=2)

In [2]:
import networkx as nx

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import Sequential

In [58]:
device = torch.device('cpu')

In [59]:
import torch_geometric.transforms as T
import torch_cluster
import torch_geometric

from torch_geometric.nn import NNConv, GCNConv, GraphConv
from torch_geometric.nn import PointConv, EdgeConv, SplineConv

In [60]:
G = nx.dodecahedral_graph()
list(nx.bfs_edges(G, 0))

[(0, 1),
 (0, 19),
 (0, 10),
 (1, 2),
 (1, 8),
 (19, 18),
 (19, 3),
 (10, 9),
 (10, 11),
 (2, 6),
 (8, 7),
 (18, 17),
 (3, 4),
 (9, 13),
 (11, 12),
 (6, 5),
 (7, 14),
 (17, 16),
 (5, 15)]

In [64]:
graph_init_state_nn = nn.Linear(5, 20)

In [65]:
class GraphRNN(nn.Module):
    def __init__(self, m):
        super().__init__()
        self.m = m
        self.node_rnn = nn.GRUCell(20, 20)
        self.edge_nn = nn.Sequential(nn.Linear(20, self.m), nn.Sigmoid())
        self.edge_prob = nn.Sequential(nn.Linear(20, 1), nn.Sigmoid())
        self.feature_generator = nn.Linear(20, 5)
        
    def forward(self, graph_state, t):
        nodes = [graph_state]
        edges = [torch.zeros(1, self.m).to(graph_state.device)]
        for i in range(t):
            graph_state = self.node_rnn(graph_state, edges[i])
            edge_state = self.edge_nn(graph_state)
            nodes.append(graph_state)
            edges.append(edge_state)
            
        return nodes, edges

In [66]:
mod = GraphRNN(20).to(device)

In [72]:
%%time
a, b = mod(torch.rand(1, 20).to(device), 1000)

CPU times: user 116 ms, sys: 4 ms, total: 120 ms
Wall time: 116 ms


In [74]:
b[1]

tensor([[0.4841, 0.5183, 0.5145, 0.4603, 0.5583, 0.4359, 0.4766, 0.5674, 0.5381,
         0.4262, 0.5093, 0.5028, 0.5379, 0.4970, 0.4726, 0.4847, 0.5536, 0.4274,
         0.4943, 0.5764]], grad_fn=<SigmoidBackward>)